In [14]:
import pandas as pd

def load_and_preprocess():
    try:
        # 데이터 로드
        df_std = pd.read_csv('team_standings_basic_2020_2025.csv')
        df_pit_war = pd.read_csv('KBO_Team_Pit_Yearly_Summary.csv')
        df_pit_basic = pd.read_csv('team_pitcher_basic_2020_2025.csv')
        df_bat_war = pd.read_csv('KBO_팀별_연도별_타격요약_통합.csv')
        df_bat_basic = pd.read_csv('team_hitter_basic_2020_2025.csv')
        df_def = pd.read_csv('team_defense_basic_2020_2025.csv')
        df_run = pd.read_csv('team_baserunning_basic_2020_2025.csv')
    except FileNotFoundError as e:
        print(f"❌ 파일을 찾을 수 없습니다: {e}")
        return None

    team_map = {'SK': 'SSG', '우리': '키움', '히어로즈': '키움', '넥센': '키움'}
    
    def clean(df, year_col='year', team_col='team'):
        df = df.rename(columns={year_col: '연도', team_col: 'Team'})
        df['Team'] = df['Team'].str.strip().replace(team_map)
        df['연도'] = df['연도'].astype(int)
        df = df[(df['연도'] >= 2020) & (df['연도'] <= 2025)]
        return df

    # 데이터 정제
    df_std = clean(df_std, 'year', 'team')
    df_pit_war = clean(df_pit_war, 'Year', 'Team')
    df_pit_basic = clean(df_pit_basic, 'year', 'team')
    df_bat_war = clean(df_bat_war, 'Year', 'Team')
    df_bat_basic = clean(df_bat_basic, 'year', 'team')
    df_def = clean(df_def, 'year', '팀명')
    df_run = clean(df_run, 'year', '팀명')

    # 데이터 병합
    master = df_std[['연도', 'rank', 'Team', 'win_rate']].rename(columns={'rank': '순위', 'win_rate': '승률'})
    master = pd.merge(master, df_pit_war[['연도', 'Team', 'WAR', 'FIP']], on=['연도', 'Team'], how='left').rename(columns={'WAR': 'WAR_pitcher'})
    master = pd.merge(master, df_pit_basic[['연도', 'Team', 'h_b1_ERA', 'h_b1_WHIP']], on=['연도', 'Team'], how='left').rename(columns={'h_b1_ERA': 'ERA', 'h_b1_WHIP': 'WHIP'})
    master = pd.merge(master, df_bat_war[['연도', 'Team', 'WAR']], on=['연도', 'Team'], how='left').rename(columns={'WAR': 'WAR_hitter'})
    master = pd.merge(master, df_bat_basic[['연도', 'Team', 'h_b1_R', 'h_b2_OPS']], on=['연도', 'Team'], how='left').rename(columns={'h_b1_R': '득점', 'h_b2_OPS': 'OPS'})
    master = pd.merge(master, df_def[['연도', 'Team', 'E', 'FPCT']], on=['연도', 'Team'], how='left').rename(columns={'E': '실책', 'FPCT': 'F%'})
    master = pd.merge(master, df_run[['연도', 'Team', 'SB%']], on=['연도', 'Team'], how='left')

    # 종합 지표 및 개별 지표 반올림 처리
    master['WAR_total'] = (master['WAR_pitcher'] + master['WAR_hitter']).round(3)
    
    # [핵심 수정] FIP를 소수점 둘째 자리까지 반올림
    master['FIP'] = master['FIP'].round(2)
    
    # 컬럼 순서 지정
    final_cols = [
        '연도', '순위', 'Team', '승률', 'WAR_pitcher', 'WAR_hitter', 'WAR_total', 
        'ERA', 'FIP', 'WHIP', 
        '득점', 'OPS', 
        '실책', 'F%', 
        'SB%'
    ]
    
    master = master.sort_values(['연도', '순위']).reset_index(drop=True)
    return master[final_cols].round(3) # 전체 데이터는 셋째 자리 유지하되 FIP만 위에서 2단위로 고정

# --- 출력 섹션 ---
final_master = load_and_preprocess()

if final_master is not None:
    final_master.to_csv('KBO_Season_Master_Data_Final.csv', index=False, encoding='utf-8-sig')

    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', 1000)
    pd.set_option('display.colheader_justify', 'center')
    pd.set_option('display.unicode.east_asian_width', True) 

    print("\n[ KBO 2020 - 2025 MASTER DATA ]")
    
    years = sorted(final_master['연도'].unique())
    
    for year in years:
        line = "=" * 145
        print(f"\n{line}")
        print(f" {year} SEASON ".center(145, " "))
        print(line)
        
        year_data = final_master[final_master['연도'] == year]
        print(year_data.to_string(index=False, justify='center'))
        
    print(f"\n" + "=" * 145)

else:
    print("❌ 데이터 통합 과정에서 오류가 발생했습니다.")


[ KBO 2020 - 2025 MASTER DATA ]

                                                                   2020 SEASON                                                                   
 연도  순위 Team  승률  WAR_pitcher  WAR_hitter  WAR_total  ERA  FIP  WHIP  득점   OPS  실책   F%   SB%
 2020    1    NC 0.601    24.94        34.65      59.59    4.58 4.85  1.42  888  0.828   87  0.983 72.7
 2020    2  두산 0.564    20.47        30.38      50.85    4.31 4.23  1.46  816  0.792   85  0.984 70.4
 2020    3    KT 0.566    24.62        31.24      55.86    4.54 4.80  1.46  813  0.794  102  0.981 67.9
 2020    4    LG 0.564    19.97        27.56      47.53    4.37 4.52  1.42  802  0.777   80  0.985 68.0
 2020    5  키움 0.559    21.20        25.89      47.09    4.39 4.54  1.39  759  0.763  112  0.980 80.7
 2020    6   KIA 0.507    15.45        16.30      31.75    5.13 4.59  1.54  724  0.755  100  0.981 65.3
 2020    7  롯데 0.497    22.20        19.50      41.70    4.64 4.54  1.41  750  0.761   94  0.982 74.4
 202